<a href="https://colab.research.google.com/github/E-tech-coder/DataScienceCapstoneProject/blob/Michi/RuleBasedModel-Michi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Rule based model (baseline)**

In [ ]:
!git clone https://github.com/E-tech-coder/DataScienceCapstoneProject.git

Cloning into 'DataScienceCapstoneProject'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 103 (delta 57), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 1.16 MiB | 6.94 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
%cd DataScienceCapstoneProject
!git checkout Michi

/content/DataScienceCapstoneProject/DataScienceCapstoneProject/DataScienceCapstoneProject/DataScienceCapstoneProject/DataScienceCapstoneProject
Branch 'Michi' set up to track remote branch 'Michi' from 'origin'.
Switched to a new branch 'Michi'


In [ ]:
!ls

Data_Exploration.ipynb	linkedin-cvs-annotated.json	   seniority-v2.csv
department-v2.csv	linkedin-cvs-not-annotated.json
df_profiles_cleansed	linkedin_experience_annotated.csv


Data sets required for the rule-based model:


*   df_profiles_cleansed: cleaned data set for testing/evaluation
*   department-v2 & seniority-v2: to define the baseline model



In [ ]:
import pandas as pd
df_profiles_cleansed = pd.read_csv("df_profiles_cleansed")
df_profiles_cleansed.head(15)

,organization,position,startDate,endDate,status,department,seniority,person_id,job_count,job_duration_years
0,Depot4Design GmbH,Prokurist,2019-08,2025-12,ACTIVE,Other,Management,0,6,6.339726
1,Depot4Design GmbH,CFO,2019-07,2025-12,ACTIVE,Other,Management,0,6,6.424658
2,Depot4Design GmbH,Betriebswirtin,2019-07,2025-12,ACTIVE,Other,Professional,0,6,6.424658
3,Depot4Design GmbH,Prokuristin,2019-07,2025-12,ACTIVE,Other,Management,0,6,6.424658
4,Depot4Design GmbH,CFO,2019-07,2025-12,ACTIVE,Other,Management,0,6,6.424658
5,Nagel Car Group,Buchhalterin,2000-05,2019-06,INACTIVE,Other,Professional,0,6,19.095890
6,Computer Solutions,Solutions Architect,2024-03,2025-12,ACTIVE,Information Technology,Professional,1,8,1.753425
7,Computer Solutions,Senior Network Engineer,2019-07,2024-03,INACTIVE,Information Technology,Senior,1,8,4.671233
8,Texas A&M University-Corpus Christi,Manager of Network Services,2017-02,2019-07,INACTIVE,Information Technology,Professional,1,8,2.410959
9,Texas A&M University-Corpus Christi,Infrastructure Administrator II,2015-06,2017-02,INACTIVE,Information Technology,Professional,1,8,1.673973


In [ ]:
df_department = pd.read_csv("department-v2.csv")
df_department.head(15)

,text,label
0,Adjoint directeur communication,Marketing
1,Advisor Strategy and Projects,Project Management
2,Beratung & Projekte,Project Management
3,Beratung & Projektmanagement,Project Management
4,Beratung und Projektmanagement kommunale Partner,Project Management
5,Cadre marketing digital,Marketing
6,Chargé de communication,Marketing
7,Chargé de communication digitale,Marketing
8,Chargé de communication et marketing,Marketing
9,Chargé de Webmarketing SEO/SEA,Marketing


In [ ]:
df_seniority = pd.read_csv("seniority-v2.csv")
df_seniority.head(15)

,text,label
0,Analyst,Junior
1,Analyste financier,Junior
2,Anwendungstechnischer Mitarbeiter,Junior
3,Application Engineer,Senior
4,Applications Engineer,Senior
5,Architecte SI - Chef de projet Applicatif,Lead
6,Associate,Junior
7,Associate - Research,Junior
8,Associate Partner,Junior
9,Associate Recruiter,Junior


Rule based model: defining current job -> Rule: ACTIVE jobs

In [ ]:
#parsing date -> Date values are converted to datetime so that ‘latest job’ can be sorted correctly
df_profiles_cleansed["startDate"] = pd.to_datetime(df_profiles_cleansed["startDate"], errors="coerce")
df_profiles_cleansed["endDate"] = pd.to_datetime(df_profiles_cleansed["endDate"], errors="coerce")

#current job -> only one active job per person
  #Rule: If ACTIVE exists → take the ACTIVE job with the latest startDate; Otherwise → take the INACTIVE job with the latest endDate
def select_current_job(group):
    active = group[group["status"] == "ACTIVE"].copy()
    if len(active) > 0:
        return active.sort_values("startDate", ascending=False).iloc[0]

    inactive = group[group["status"] != "ACTIVE"].copy()
    return inactive.sort_values("endDate", ascending=False).iloc[0]

current_df = (
    df_profiles_cleansed
    .groupby("person_id", as_index=False)
    .apply(select_current_job)
    .reset_index(drop=True)
)

current_df.head()

/tmp/ipython-input-3174362220.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(select_current_job)


,organization,position,startDate,endDate,status,department,seniority,person_id,job_count,job_duration_years
0,Depot4Design GmbH,Prokurist,2019-08-01,2025-12-01,ACTIVE,Other,Management,0,6,6.339726
1,Computer Solutions,Solutions Architect,2024-03-01,2025-12-01,ACTIVE,Information Technology,Professional,1,8,1.753425
2,Udo Weber,Medizintechnik Beratung,2025-01-01,2025-12-01,ACTIVE,Consulting,Professional,2,3,0.915068
3,Grupo Viajes Kontiki.,Director expansión de negocio.,2024-09-01,2025-12-01,ACTIVE,Business Development,Director,3,7,1.249315
4,Himmelstalunds Utbildningscentrum,"APL-ansvarig, samordning",2019-08-01,2025-12-01,ACTIVE,Administrative,Lead,4,8,6.339726


In [ ]:
print(df_profiles_cleansed.columns)
print(df_seniority.columns)
print(df_department.columns)

Index(['organization', 'position', 'startDate', 'endDate', 'status',
       'department', 'seniority', 'person_id', 'job_count',
       'job_duration_years'],
      dtype='object')
Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


In [ ]:
#separation of the column names for department and seniority data sets
df_seniority = df_seniority.rename(columns={"text": "title_s", "label": "seniority"})
df_department = df_department.rename(columns={"text": "title_d", "label": "department"})

In [ ]:
#text normalization
import re

def norm(text):
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
#dictionaries (Fallback)
sen_map = dict(zip(df_seniority["title_s"].map(norm), df_seniority["seniority"]))
dep_map = dict(zip(df_department["title_d"].map(norm), df_department["department"]))

print("sen_map:", len(sen_map), "entries")
print("dep_map:", len(dep_map), "entries")

sen_map: 9413 entries
dep_map: 10132 entries


**Rules for seniority und departments**




In [ ]:
#seniority rules
def predict_seniority(title):
    t = norm(title)

    #hard rules
    if any(k in t for k in [
        "ceo","cfo","cto","cmo","coo","chief",
        "director","vice president","vp","head of","head ",
        "owner","co-owner","founder","cofounder","co-founder",
        "prokurist","prokuristin","geschäftsführer","geschaeftsfuehrer"]):
        return "Management", "keyword_mgmt"

    if "lead" in t:
        return "Lead", "keyword_lead"

    if any(k in t for k in ["senior","sr.","principal","staff"]):
        return "Senior", "keyword_senior"

    if any(k in t for k in ["intern","internship","trainee","working student","werkstudent","junior","apprentice","auszubild"]):
        return "Entry", "keyword_entry"

    #fallback exact match in label list
    if t in sen_map:
        return sen_map[t], "list_exact"

    return "Professional", "default"

In [ ]:
#department rules
def predict_department(title, organization=None):
    t = norm(title)

    #hard rules
    if any(k in t for k in ["marketing","growth","brand","content","seo","sea","social media","communications","kommunikation","cmo"]):
        return "Marketing", "keyword_marketing"

    if any(k in t for k in ["software","developer","engineer","architect","devops","cloud","data scientist","data engineer",
                         "network","systems administrator","administrator","it"]):
        return "Information Technology", "keyword_it"

    if any(k in t for k in ["finance","accountant","buchhalter","controller","controlling","fp&a","audit","auditor","treasury","cfo"]):
        return "Finance", "keyword_finance"

    if any(k in t for k in ["sales","account executive","business development","bd","vertrieb","key account"]):
        return "Sales", "keyword_sales"

    if any(k in t for k in ["logistics","supply","driver","delivery","warehouse","dispatch","transport"]):
        return "Logistics", "keyword_logistics"

    # fallback exact match in label list
    if t in dep_map:
        return dep_map[t], "list_exact"

    return "Other", "default"

**Evaluation**

In [ ]:
#predictions of current_df

rows = []
for _, r in current_df.iterrows():
    dep_pred, dep_reason = predict_department(r["position"], r.get("organization"))
    sen_pred, sen_reason = predict_seniority(r["position"])

    rows.append({
        "person_id": r["person_id"],
        "title": r["position"],
        "org": r["organization"],
        "true_department": r["department"],
        "pred_department": dep_pred,
        "dep_reason": dep_reason,
        "true_seniority": r["seniority"],
        "pred_seniority": sen_pred,
        "sen_reason": sen_reason,
    })

df_pred = pd.DataFrame(rows)

#accuracy
dept_acc = (df_pred["pred_department"] == df_pred["true_department"]).mean()
sen_acc  = (df_pred["pred_seniority"] == df_pred["true_seniority"]).mean()

print(f"Department accuracy: {dept_acc:.2%}")
print(f"Seniority accuracy:  {sen_acc:.2%}")
df_pred.head(10)

Department accuracy: 55.85%
Seniority accuracy:  63.10%


,person_id,title,org,true_department,pred_department,dep_reason,true_seniority,pred_seniority,sen_reason
0,0,Prokurist,Depot4Design GmbH,Other,Other,default,Management,Management,keyword_mgmt
1,1,Solutions Architect,Computer Solutions,Information Technology,Information Technology,keyword_it,Professional,Professional,default
2,2,Medizintechnik Beratung,Udo Weber,Consulting,Other,default,Professional,Professional,default
3,3,Director expansión de negocio.,Grupo Viajes Kontiki.,Business Development,Other,default,Director,Management,keyword_mgmt
4,4,"APL-ansvarig, samordning",Himmelstalunds Utbildningscentrum,Administrative,Other,default,Lead,Professional,default
5,5,Kaufmännischer Leiter,FSD Fahrzeugsystemdaten GmbH,Sales,Information Technology,keyword_it,Lead,Lead,list_exact
6,6,Lab-Supervisor,MacDermid Enthone Industrial Solutions,Other,Other,default,Lead,Professional,default
7,7,Human Resources Generalist,Boise Cascade Company,Human Resources,Other,default,Professional,Professional,default
8,8,Managing Director,PhoboSys GmbH,Other,Other,default,Management,Management,keyword_mgmt
9,9,Meister,*,Other,Other,default,Lead,Professional,default


In [ ]:
#error analysis

dept_errors = df_pred[df_pred["pred_department"] != df_pred["true_department"]]
sen_errors  = df_pred[df_pred["pred_seniority"] != df_pred["true_seniority"]]

print("Department errors:", len(dept_errors))
print("Seniority errors:", len(sen_errors))

#department errors
dept_errors[["title","true_department","pred_department","dep_reason"]].head(15)

Department errors: 268
Seniority errors: 224


,title,true_department,pred_department,dep_reason
2,Medizintechnik Beratung,Consulting,Other,default
3,Director expansión de negocio.,Business Development,Other,default
4,"APL-ansvarig, samordning",Administrative,Other,default
5,Kaufmännischer Leiter,Sales,Information Technology,keyword_it
7,Human Resources Generalist,Human Resources,Other,default
10,Applikation Analyst Senior,Information Technology,Other,default
13,Leistungssachbearbeiter SGB II,Other,Information Technology,keyword_it
14,energy and sustainability consultant bei,Consulting,Information Technology,keyword_it
22,Bereichsleiter Maschinen,Other,Information Technology,keyword_it
26,Consultor de vendas,Sales,Other,default


In [ ]:
#seniority errors

sen_errors[["title","true_seniority","pred_seniority","sen_reason"]].head(15)

,title,true_seniority,pred_seniority,sen_reason
3,Director expansión de negocio.,Director,Management,keyword_mgmt
4,"APL-ansvarig, samordning",Lead,Professional,default
6,Lab-Supervisor,Lead,Professional,default
9,Meister,Lead,Professional,default
15,Purchasing Manager,Professional,Senior,list_exact
22,Bereichsleiter Maschinen,Lead,Professional,default
25,Unternehmensinhaber,Management,Professional,default
31,Director,Director,Management,keyword_mgmt
36,Head of HR/CHRO,Lead,Management,keyword_mgmt
38,Office Associate,Junior,Professional,default


The rule-based baseline achieves 63% accuracy for seniority prediction and 56% for department prediction.
The performance gap reflects the higher semantic ambiguity of department classification compared to seniority, especially across languages and organizational contexts.
Error analysis shows systematic weaknesses for roles such as consulting, HR, and administrative positions, highlighting the limitations of purely keyword-based approaches and motivating the use of embedding-based and supervised models.

**Pipeline:**

In [ ]:
def pipeline(job_title, organization=None):
    dep, _ = predict_department(job_title, organization)
    sen, _ = predict_seniority(job_title)
    return {"department": dep, "seniority": sen}

In [ ]:
pipeline("CMO")

{'department': 'Marketing', 'seniority': 'Management'}

In [ ]:
tests = ["CMO", "CFO", "Senior Network Engineer", "Marketing Intern", "Human Resources Generalist", "Consultant"]
for t in tests:
    print(t, "->", pipeline(t))

CMO -> {'department': 'Marketing', 'seniority': 'Management'}
CFO -> {'department': 'Finance', 'seniority': 'Management'}
Senior Network Engineer -> {'department': 'Information Technology', 'seniority': 'Senior'}
Marketing Intern -> {'department': 'Marketing', 'seniority': 'Entry'}
Human Resources Generalist -> {'department': 'Other', 'seniority': 'Professional'}
Consultant -> {'department': 'Consulting', 'seniority': 'Senior'}


In [ ]:
df_pred_v2.sample(10)

,person_id,title,true_department,pred_department,dep_reason,true_seniority,pred_seniority,sen_reason
472,474,im (Un-)Ruhestand,Other,Other,default,Professional,Professional,default
9,9,Meister,Other,Other,default,Lead,Professional,default
488,490,Geschäftsführer,Other,Other,default,Management,Management,keyword_mgmt
174,174,Lead Consultant,Consulting,Other,default,Lead,Lead,keyword_lead
52,52,Chief Revenue Officer (CRO),Sales,Other,default,Management,Management,keyword_mgmt
45,45,Reporter,Other,Other,default,Professional,Professional,default
367,369,Head of Sales DACH,Sales,Sales,keyword_sales,Lead,Management,keyword_mgmt
468,470,Key Account / Project Manager,Other,Sales,keyword_sales,Lead,Professional,default
93,93,Key-Account-Manager,Sales,Sales,list_exact,Professional,Senior,list_exact
105,105,Lehrbeauftragter,Consulting,Other,default,Lead,Professional,default
